## BoxScores Updating

PLAYER BOXSCORES

Need to Know:
- We download data by the MONTH.
- Nba.com "Months" start in october (i.e., the beginning of the season). So 10 = 1. 

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
import os
import winsound

home_folder = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1'
os.chdir(home_folder)

In [68]:
def grab_player_data2(url_list, file_folder):    
        
        # Scrape Season-Level player data from the url_list

        i = 0
        for u in url_list:
                
                driver.get(u)
                time.sleep(2)

                # if the page does not load, go to the next in the list
                try:
                        xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]'
                        elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath)))
                except:
                        print(f'{u} did not load. Moving to next url.')
                        continue

                # click "all pages"
                xpath_all = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]' 
                elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_all)))
                
                driver.find_element(by=By.XPATH, value=xpath_all).click()
                src = driver.page_source
                parser = BeautifulSoup(src, "lxml")
                table = parser.find("table", attrs = {"class":"Crom_table__p1iZz"})
                headers = table.findAll('th')
                headerlist = [h.text.strip() for h in headers[0:]] 
                row_names = table.findAll('a')                             # find rows
                row_list = [b.text.strip() for b in row_names[0:]] 
                rows = table.findAll('tr')[0:]
                player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]
                tot_cols = len(player_stats[1])                           #set the length to ignore hidden columns
                headerlist = headerlist[:tot_cols]   
                stats = pd.DataFrame(player_stats, columns = headerlist)

                # assign filename
                filename = file_folder + str(u[34:]).replace('/', '_') + '.csv'
                filename = replace_name_values2(filename)
                pd.DataFrame.to_csv(stats, filename)
                i += 1
                lu = len(url_list)
                # close driver
                print(f'{filename} Completed Successfully! {i} / {lu} Complete!')

        winsound.Beep(523, 500)

In [69]:
def replace_name_values2(filename):
        # replace values with dashes for compatibility
    filename = filename.replace('%','_')
    filename = filename.replace('=','_')
    filename = filename.replace('?','_')
    filename = filename.replace('&','_')
    filename = filename.replace('20Season_','')
    filename = filename.replace('_20Season','')
    filename = filename.replace('SeasonType_','')
    filename = filename.replace('sort_gdate_dir_-1_','')
    filename = filename.replace('SeasonYear_','')
    return filename

In [70]:
def append_the_data(folder, data_prefix, filename_selector):
    # Appending data together via folder and/or file name

    path = folder
    p = os.listdir(path)
    pf = pd.DataFrame(p)


    # filter for files that contain the filename_selector
    pf_reg = pf.loc[pf[0].astype(str).str.contains(filename_selector)] 

    appended_data = []
    for file in pf_reg[0]:
        data = pd.read_csv(folder + '/' + file)
        # if "Season" a column, drop it
        if 'Season' in data.columns:
            data = data.drop(columns = ['Season'])
        
        data['season'] = file[(file.find('20')):(file.find('20'))+4]
        data['season_type'] = np.where('Regular' in file, 'Regular', 'Playoffs')
        # add prefix to columns
        data = data.add_prefix(data_prefix)
        data.columns = data.columns.str.lower()
        appended_data.append(data)
    
    appended_data = pd.concat(appended_data)
    return appended_data

In [71]:
today = datetime.date.today()
print(today)
month = today.month
print(month)

2022-12-24
12


In [72]:
# read current boxscores, get the last date
boxscores = pd.read_csv('data/player/aggregates/Trad&Adv_box_scores_GameView.csv')
boxscores.head()

,Unnamed: 0,trad_unnamed: 0,trad_player,trad_team,trad_match up,trad_game date,trad_w/l,trad_min,trad_pts,trad_fgm,...,adv_dreb%,adv_reb%,adv_to ratio,adv_efg%,adv_ts%,adv_usg%,adv_pace,adv_pie,adv_season,adv_season_type
0,7,1,Damian Lillard,POR,POR vs. LAL,10/31/2012,W,35.0,23.0,7.0,...,10.7,4.9,16.2,44.1,56.0,32.1,95.36,16.3,2012,Regular
1,8,2,Kobe Bryant,LAL,LAL @ POR,10/31/2012,L,38.0,30.0,10.0,...,10.3,8.6,21.2,60.0,65.0,31.9,99.16,10.3,2012,Regular
2,9,3,Nicolas Batum,POR,POR vs. LAL,10/31/2012,W,40.0,26.0,9.0,...,6.1,8.2,4.8,65.6,68.1,21.1,99.17,14.8,2012,Regular
3,10,4,Al Jefferson,UTA,UTA vs. DAL,10/31/2012,W,29.0,12.0,4.0,...,36.0,23.0,6.3,36.4,47.0,18.4,97.68,15.9,2012,Regular
4,11,5,Paul Millsap,UTA,UTA vs. DAL,10/31/2012,W,33.0,13.0,5.0,...,22.6,21.1,0.0,41.7,47.2,16.1,96.97,12.9,2012,Regular


In [73]:
boxscores['date'] = pd.to_datetime(boxscores['trad_game date'])
boxscores = boxscores.sort_values(by = 'date', ascending = False)
boxscores = boxscores.reset_index(drop = True)
last_date = boxscores['date'][0]
# change to date from datetime
last_date = last_date.date()
last_date

datetime.date(2022, 12, 23)

In [74]:
# get months since oct 2022
today = datetime.date.today()
months_since_oct_2022 = (today.year - 2022) * 12 + today.month - 9
this_season_months = np.arange(1, months_since_oct_2022+1, 1)

s_17 = '2017-18'
s_18 = '2018-19'
s_19 = '2019-20'
s_20 = '2020-21'
s_21 = '2021-22'
s_22 = '2022-23'

s_16 = '2016-17'
s_15 = '2015-16'
s_14 = '2014-15'
s_13 = '2013-14'
s_12 = '2012-13'
s_11 = '2011-12'

yearz = ['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011']
s_years = [s_22, s_21, s_20, s_19, s_18, s_17, s_16, s_15, s_14, s_13, s_12, s_11]

#months played by year -- the months that HAD GAMES during said season
# October is counted as Month #1 --- Most should go 1-7. 
months_2022 = this_season_months
months_2021 = ['1', '2','3', '4', '5', '6', '7']
months_2020 = ['3', '4', '5', '6', '7','8']
months_2019 = ['1', '2','3', '4', '5', '6']
months_2018 = ['1', '2','3', '4', '5', '6', '7']
months_2017 = ['1', '2','3', '4', '5', '6', '7']
months_2016 = ['1', '2','3', '4', '5', '6', '7']
months_2015 = ['1', '2','3', '4', '5', '6', '7']
months_2014 = ['1', '2','3', '4', '5', '6', '7']
months_2013 = ['1', '2','3', '4', '5', '6', '7']
months_2012 = ['1', '2','3', '4', '5', '6', '7']
months_2011 = ['1', '2','3', '4', '5', '6', '7']

years = ['2022-23','2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12']
types = ['boxscores-advanced', 'boxscores-traditional']
season_types = ['Regular%20Season', 'Playoffs']


In [75]:
print(this_season_months)
current_month = max(this_season_months)
print(f'Current Month is {current_month}')

[1 2 3]
Current Month is 3


In [76]:
trad_url_update = ('https://www.nba.com/stats/players/boxscores-traditional/?Season=2022-23&sort=gdate&dir=-1&Month='+str(current_month) +'&SeasonType=Regular%20Season')
adv_url_update = ('https://www.nba.com/stats/players/boxscores-advanced/?Season=2022-23&sort=gdate&dir=-1&Month='+str(current_month) +'&SeasonType=Regular%20Season')

In [77]:
def get_all_urls():
    # List all months to scrape
    urls1 = []

    for m in months_2022:
        for t in types:
            url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2022-23&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
            urls1.append(url)

    for m in months_2021:
        for t in types: 
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2021-22&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2020:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2020-21&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2019: 
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2019-20&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2018:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2018-19&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2017:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2017-18&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2016:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2016-17&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2015:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2015-16&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2014:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2014-15&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2013:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2013-14&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    for m in months_2012:
        for t in types:
                url = 'https://www.nba.com/stats/players/' + str(t) +'/?Season=2012-13&sort=gdate&dir=-1&Month='+str(m) +'&SeasonType=Regular%20Season'
                urls1.append(url)

    return urls1

In [78]:
print(today)
print(last_date)

2022-12-24
2022-12-23


In [80]:
# if the last_date is before yesterday, then scrape the new data
yesterday = today - datetime.timedelta(days=1)

if last_date < yesterday:
    driver = webdriver.Chrome()
    # update this month's data
    trad_url_update = ('https://www.nba.com/stats/players/boxscores-traditional/?Season=2022-23&sort=gdate&dir=-1&Month='+str(current_month) +'&SeasonType=Regular%20Season')
    adv_url_update = ('https://www.nba.com/stats/players/boxscores-advanced/?Season=2022-23&sort=gdate&dir=-1&Month='+str(current_month) +'&SeasonType=Regular%20Season')
    urls2 = [trad_url_update, adv_url_update]
    # scrape trad and adv updates
    grab_player_data2(urls2, 'data/player/box_scores/')

else:
    print('Data is Updated')
    
    

Data is Updated


#### Append

In [63]:
all_boxes_trad = append_the_data('data/player/box_scores/', 'trad_', 'traditional')
all_boxes_adv = append_the_data('data/player/box_scores/', 'adv_', 'advanced')

all_boxes_trad.shape

(261950, 28)

In [64]:
all_boxes = pd.merge(all_boxes_trad,all_boxes_adv,
                        left_on=['trad_player', 'trad_team', 'trad_season', 'trad_season_type', 'trad_match up', 'trad_game date'],
                        right_on=['adv_player', 'adv_team', 'adv_season', 'adv_season_type', 'adv_match up', 'adv_game date'],
                        how='left')
all_boxes

,trad_unnamed: 0,trad_player,trad_team,trad_match up,trad_game date,trad_w/l,trad_min,trad_pts,trad_fgm,trad_fga,...,adv_dreb%,adv_reb%,adv_to ratio,adv_efg%,adv_ts%,adv_usg%,adv_pace,adv_pie,adv_season,adv_season_type
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,Regular
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,Regular
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,Regular
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,Regular
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262347,3536,Killian Hayes,DET,DET vs. DAL,12/01/2022,W,33.0,22.0,10.0,13.0,...,12.9,7.0,8.7,84.6,84.6,20.3,93.14,17.0,2022,Regular
262348,3537,Christian Wood,DAL,DAL @ DET,12/01/2022,L,35.0,25.0,10.0,13.0,...,20.6,12.5,9.5,80.8,79.9,25.0,96.46,18.9,2022,Regular
262349,3538,Marvin Bagley III,DET,DET vs. DAL,12/01/2022,W,34.0,19.0,7.0,10.0,...,28.1,21.3,6.3,75.0,75.2,17.7,93.97,15.1,2022,Regular
262350,3539,Tim Hardaway Jr.,DAL,DAL @ DET,12/01/2022,L,40.0,26.0,9.0,20.0,...,7.7,6.8,0.0,60.0,59.7,24.4,96.64,9.9,2022,Regular


In [65]:
all_boxes = all_boxes.dropna(subset=['trad_player'])

In [66]:
all_boxes.to_csv('data/player/aggregates/Trad&Adv_box_scores_GameView.csv')